In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
import pandas as pd

In [ ]:
def win_game(p):
    """
    PURPOSE: Calculates the probability of the server winning a game
    PARAMETERS:
        p: The probability of the server winning point
        cacheArray: stores previosuly calculated probabilities to speed up things for larger simulations
    RETURNS: The probability of winning the game
    """
    win_game=0
    for i in range(4,7):
        win_game+=binomial_prob(i,6,p)
    win_game+=binomial_prob(3,6,p)*win_from_deuce(p)
    return win_game

def binomial_prob(x,n,p):
    """ 
    PURPOSE: Calculates the probability of the event ocurring
    PARAMETERS:
        x: The number of successes
        n: The number of trials
        p: The probability of success for a given trial
    RETURNS: The probability of getting x successes in n trials
    """
    return math.comb(n,x)*np.power(p,x)*np.power(1-p,n-x)

def win_from_deuce(p):
    """
    PURPOSE: Calculates the probability of the server winning a game after reachind deuce
    PARAMETERS:
        p: The probability of the server winning point
    RETURNS: The probability of winning the game
    """
    return p**2/(p**2+(1-p)**2)



In [ ]:
def sim_games(server_win_prob,n):
    """
    PURPOSE: Simulates a series of games and calculates what percent the server won
    PARAMETERS:
        server_win_prob: The probability of the server winning a point
        n:             The number of games to simulate
    RETURNS: The percent of simulated games that were won by the server
    """
    score_dict = {
      "Server": 0,
      "Returner": 0
    }
    for i in range (0,n):
        if sim_game(server_win_prob)==1:
            score_dict["Server"]+=1
        else: 
            score_dict["Returner"]+=1
    return score_dict["Server"]/n
def sim_game(server_win_prob):
    """
    PURPOSE: Simulates a game
    PARAMETERS:
        server_win_prob: The probability of the server winning a point
    RETURNS: 1 if the server won the game, 0 if the returner won the game
    """
    point=0
    score_dict = {
      "Server": 0,
      "Returner": 0
    }
    while True:
        point=point+1
        if sim_event(server_win_prob)==1:
            score_dict["Server"]+=1
        else: 
            score_dict["Returner"]+=1
        if score_dict["Server"]>=4 and score_dict["Server"]-score_dict["Returner"]>=2:
            return 1
        if score_dict["Returner"]>=4 and score_dict["Returner"]-score_dict["Server"]>=2:
            return 0
def sim_event(server_win_prob):
    """
    PURPOSE: Simulates a single event, usually a point or a game.
    PARAMETERS:
        server_win_prob: The probability of the server winning the point
    RETURNS: 1 if the server won the point, 0 if the returner won the point
    """
    if server_win_prob>np.random.rand():
        return 1
    return 0

In [ ]:
n1=1000
n2=10000
resolution=101
prob_win_point=np.linspace(0, 1, resolution)
prob_win_game_1000= np.empty([resolution])
prob_win_game_10000= np.empty([resolution])
#Simulate 1,000 and 10,000 games for each value of p
for i in range(0,len(prob_win_point)):
    prob_win_game_1000[i]=sim_games(prob_win_point[i],n1)
    prob_win_game_10000[i]=sim_games(prob_win_point[i],n2)


In [ ]:
plt.plot(prob_win_point, prob_win_game_1000, color='blue', label = '1,000 Simulations')
plt.plot(prob_win_point, prob_win_game_10000, color='green', label = '10,000 Simulations')
plt.plot(prob_win_point, win_game(prob_win_point), color='red', label='Analytical Solution')

plt.xlabel('Probability of Winning a Point')
plt.ylabel('Probability of Winning a Game')
plt.title('Probability of Winning a Game Given Probability of Winning a Point (Numeric vs Analytical Solutions)',pad=20)
plt.legend()
plt.show()

In [ ]:
def sim_sets(server_prob_win_point_side1,server_prob_win_point_side2,n):
    """
    PURPOSE: Simulates a series of sets and calculates what percent the first server won
    PARAMETERS:
        server_prob_win_point_side1: The probability of the server winning a point when serving from side 1
        server_prob_win_point_side2: The probability of the server winning a point when servering from side 2
        n:                  The number of sets to simulate
    RETURNS: The percent of simulated sets that were won by the first server
    """
    if server_prob_win_point_side1==1 and server_prob_win_point_side2==1:
        return np.NaN
    if server_prob_win_point_side1==0 and server_prob_win_point_side2==0:
        return np.NaN
    score_dict = {
        0:0,
        1:0,
        2:0
    }
    server_prob_win_game_side1=win_game(server_prob_win_point_side1)
    server_prob_win_game_side2=win_game(server_prob_win_point_side2)
    for i in range (0,n):
        result=sim_set(server_prob_win_game_side1,server_prob_win_game_side2)
        if result==0:
            if sim_tiebreak(server_prob_win_point_side1,server_prob_win_point_side2)==1:
                result=1
            else:
                result=2
        if result==1:
            score_dict[1]+=1
        elif result==2:
            score_dict[2]+=1
    return score_dict[1]/n

def sim_set(server_prob_win_game_side1,server_prob_win_game_side2):
    """
    PURPOSE: Simulates a set, up to the tiebreadker.
    PARAMETERS:
        server_prob_win_point_side1: The probability of the server winning a point when serving from side 1
        server_prob_win_point_side2: The probability of the server winning a point when servering from side 2
    RETURNS: 1 if the server won the set, 2 if the returner won the set and 0 if the set went to a tiebreaker
    """
    game=0
    server=1
    side=1
    score_dict = {
        1:0,
        2:0
    }
    while True:
        game=game+1
        if side==1:
            server_win_prob=server_prob_win_game_side1
        else:
            server_win_prob=server_prob_win_game_side2
        if sim_event(server_win_prob):
            score_dict[server]+=1
        else:
            score_dict[server%2+1]+=1
        if score_dict[1]>=6 and score_dict[1]-score_dict[2]>=2:
            return 1
        if score_dict[2]>=6 and score_dict[2]-score_dict[1]>=2:
            return 2
        if score_dict[2]==6 and score_dict[1]==6:
            return 0
        server=server%2+1
        side=side%2+1
        if game%2==1:
            side=side%2+1 
def sim_tiebreak(server_prob_win_point_side1,server_prob_win_point_side2):
    """
    PURPOSE: Simulates a tiebreaker
    PARAMETERS:
        server_prob_win_point_side1: The probability of the server winning a point when serving from side 1
        server_prob_win_point_side2: The probability of the server winning a point when servering from side 2
    RETURNS: 1 if the server won the tiebreaker, 0 if the returner won the tiebreaker
    """
    point=0
    server=1
    side=1
    score_dict = {
        1:0,
        2:0
    }
    while True:
        point=point+1
        if side==1:
            server_win_prob=server_prob_win_point_side1
        else:
            server_win_prob=server_prob_win_point_side2

        if sim_event(server_win_prob):
            score_dict[server]+=1
        else:
            score_dict[server%2+1]+=1
        if score_dict[1]>=7 and score_dict[1]-score_dict[2]>=2:
            return 1
        if score_dict[2]>=7 and score_dict[2]-score_dict[1]>=2:
            return 2
        if point%2==1:
            server=server%2+1
            side=side%2+1
        if point%6==0:
            side=side%2+1

def runSimTiebreak(server_prob_win_point_side1,server_prob_win_point_side2,n):
    """
    DEPRECATED
    PURPOSE: Simulates a series of tiebreakers and calculates what percent the first server won
    PARAMETERS:
        server_prob_win_point_side1: The probability of the server winning a point when serving from side 1
        server_prob_win_point_side2: The probability of the server winning a point when servering from side 2
        n:                  The number of tiebreakers to simulate
    RETURNS: The percent of simulated tiebreakers that were won by the first server
    """
    if server_prob_win_point_side1==1 and server_prob_win_point_side2==1:
        return np.NaN
    if server_prob_win_point_side1==0 and server_prob_win_point_side2==0:
        return np.NaN
    score_dict = {
        1:0,
        2:0
    }
    for i in range (0,n):
        score_dict[sim_tiebreak(server_prob_win_point_side1,server_prob_win_point_side2)]+=1
    return score_dict[1]/n

In [ ]:
resolution=21
n=100000
server_prob_win_point_side1 = np.linspace(0, 1, resolution).round(2)
server_prob_win_point_side2 = np.linspace(0, 1, resolution).round(2)
P1, P2 = np.meshgrid(server_prob_win_point_side1, server_prob_win_point_side2)
server_probability_array=np.empty([resolution,resolution])
returner_probability_array=np.empty([resolution,resolution])
for row in range (0,resolution):
    for col in range (0,resolution):
        p1=P1[row][col]
        p2=P2[row][col]
        server_prob_win_set=sim_sets(p1, p2,n)
        server_probability_array[row][col]=server_prob_win_set
        returner_probability_array[row][col]=1-server_prob_win_set
        

In [ ]:
def plot_df(df,xlabel,ylabel,title):
    """
    PURPOSE: Helper function to plot heatmaps
    PARAMETERS:
        df: The dataframe to plot
        xlabel: The xaxis label
        tlabel: The yaxis label
        title:  The plot title
    """
    sns.set()
    ax = sns.heatmap(df)
    ax.invert_yaxis()
    tick_positions=np.linspace(0, resolution, 11).round(2)
    tick_labels=np.linspace(0, 1, 11).round(2)
    ax.set_xticks(tick_positions)
    ax.set_xticklabels(tick_labels)
    ax.set_yticks(tick_positions)
    ax.set_yticklabels(tick_labels)
    if (flip_axes):
        ax.invert_yaxis()
        ax.invert_xaxis()
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title,pad=20)
    plt.show()

In [ ]:
server_df = pd.DataFrame(server_probability_array, index=server_prob_win_point_side1,columns=server_prob_win_point_side2)
xlabel='Probability of Winning a Point Serving From Side 1'
ylabel='Probability of Winning a Point Serving From Side 2'
title='Probability of Winning a Set as the First Server'

plot_df(server_df,xlabel,ylabel,title)

In [ ]:
returner_df = pd.DataFrame(np.rot90(returner_probability_array,2), index=server_prob_win_point_side1,columns=server_prob_win_point_side2)
xlabel='Probability of Winning a Point Receiving From Side 1'
ylabel='Probability of Winning a Point Receiving From Side 2'
title='Probability of Winning a Set as the First Receiver'
plot_df(returner_df,xlabel,ylabel,title)

In [ ]:
#Calculate the probability and error bounds of winning a set serving
#Given article's inital conditions if your opponent chooses to serve
n=1000000
p1=0.8
p2=0.55
simProb=sim_sets(p1,p2,n)
SD=np.sqrt(simProb*(1-simProb)/n)
print(simProb,"+/-",SD*1.96)

#Calculate the probability and error bounds of winning a set recieving
#given article's inital conditions if your opponent chooses to recieve
n=1000000
p1=0.55
p2=0.8
simProb=1-sim_sets(p1,p2,n)
SD=np.sqrt(simProb*(1-simProb)/n)
print(simProb,"+/-",SD*1.96)